In [1]:
import json
import time
import os
import sklearn_crfsuite
import re
import nereval
import pandas as pd
from preprocessing_protocols import preprocess_text

from estnltk import Text
from estnltk.taggers import NerTagger
from estnltk.taggers import WordLevelNerTagger
from estnltk.converters import text_to_json
from estnltk.converters import json_to_text
from estnltk.layer_operations import flatten
from sklearn.metrics import classification_report
from sklearn_crfsuite import metrics

from nervaluate import Evaluator

from estnltk.taggers.estner.ner_trainer import NerTrainer
from estnltk.taggers.estner.model_storage_util import ModelStorageUtil
from estnltk.core import DEFAULT_PY3_NER_MODEL_DIR

In [51]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

These files don't work because the protocols are written in a different language, which the goldstandard didn't recognise, hence have no goldstandard tags.

In [50]:
files_not_working = ['J2rva_Tyri_V22tsa_id22177_1911a.json', \
                     'J2rva_Tyri_V22tsa_id18538_1894a.json', \
                     'J2rva_Tyri_V22tsa_id22155_1911a.json', \
                     'Saare_Kihelkonna_Kotlandi_id18845_1865a.json', \
                     'P2rnu_Halliste_Abja_id257_1844a.json', \
                     'Saare_Kaarma_Loona_id7575_1899a.json', \
                     'J2rva_Tyri_V22tsa_id22266_1913a.json', \
                     'J2rva_Tyri_V22tsa_id22178_1912a.json']

In [48]:
files = {}

with open('divided_corpus.txt', 'r', encoding = 'UTF-8') as f:
    txt = f.readlines()

for fileName in txt:
    file, subdistribution = fileName.split(":")
    files[file] = subdistribution.rstrip("\n")

In [5]:
all_results = {}

for subdistribution in [1, 2, 3, 4, 5]:
    training_subdistributions = []
    for y in [1, 2, 3, 4, 5]:
        if y == subdistribution:
            subdistribution_for_testing = y
        else:
            training_subdistributions.append(y)
    
    # Getting the filenames to be trained on from the files dictionary.
    filenames = {key: value for key, value in files.items() if int(value) in training_subdistributions}
    
    # Creating training_texts from the aforementioned filenames.
    print("Valmistan ette treenimistekste.")
    start = time.time()
    training_texts = []
    for filename in filenames:
        with open('./vallakohtufailid-json-flattened/' + str(filename), 'r', encoding='UTF-8') as file:
            if filename in files_not_working:
                continue
            else:
                training_texts.append(preprocess_text(json_to_text(file.read())))
    print(f"Treenimistekstid ette valmistatud {time.time() - start} sekundiga.")
    
    # Setting up the trainer and training.
    print("\n\nAlustan nertaggeri treenimist.")
    start = time.time()
    model_dir=DEFAULT_PY3_NER_MODEL_DIR
    modelUtil = ModelStorageUtil(model_dir)
    nersettings = modelUtil.load_settings()
    trainer = NerTrainer(nersettings)
    trainer.train( training_texts, layer='gold_wordner', model_dir='test' )
    print(f"NerTagger treenitud {time.time() - start} sekundiga.")
    
    # Setting up the new trained nertagger and defining layers to be removed later on.
    nertagger = NerTagger(model_dir = 'test')
    removed_layers = ['sentences', 'morph_analysis', 'compound_tokens', 'ner', 'words', 'tokens']
    
    # Tagging the files using the new nertagger.
    print("\n\nAlustan failide taggimist.")
    start = time.time()
    for file in {key: value for key, value in files.items() if int(value) == subdistribution_for_testing}:
        with open(find(file.replace(".json", ".txt"), "./vallakohtufailid/"), 'r', encoding='UTF-8') as f:
            text = f.read()
            if file == "Tartu_V6nnu_Ahja_id3502_1882a.txt":
                text = text.replace('..', '. .')
            text = preprocess_text(Text(text))
            nertagger.tag(text)
            text.add_layer(flatten(text['ner'], 'flat_ner'))

            for x in removed_layers:
                text.pop_layer(x)
            
            text_to_json(text, file=os.getcwd() + "/vallakohtufailid-trained-nertagger/" + file)
            print(f'Täägitud fail {file}')
    print(f"Failid taggitud {time.time() - start} sekundiga.")
    
    # Chaning the tags into a readable formats for the evaluator.
    print("\n\nAlustan tulemuste ammutamist.")

    #gold = []
    #test = []
    gold_ner = []
    test_ner = []

    for file in {key: value for key, value in files.items() if int(value) == subdistribution_for_testing}:
        appendable_gold_ner = []
        appendable_test_ner = []

        if file.endswith(".json"):
            if file in files_not_working:
                continue
            else:
                with open("./vallakohtufailid-trained-nertagger/" + str(file), 'r', encoding='UTF-8') as f_test, \
                    open("./vallakohtufailid-json-flattened/" + str(file), 'r', encoding='UTF-8') as f_gold:
                        test_import = json_to_text(f_test.read())
                        gold_import = json_to_text(f_gold.read())

                        # The commented part is needed for word-level-ner.
                        '''
                        for i in range(len(gold_import['flat_gold_wordner'])):
                            tag = gold_import['flat_gold_wordner'][i].nertag[0]
                            gold.append(tag)
                        for i in range(len(test_import['flat_wordner'])):
                            tag = test_import['flat_wordner'][i].nertag[0]
                            test.append(tag)
                        '''

                        for i in range(len(gold_import['gold_ner'])):
                            ner = gold_import['gold_ner'][i]
                            label = ner.nertag
                            start = int(ner.start)
                            end = int(ner.end)
                            appendable_gold_ner.append({"label": label, "start": start, "end": end})

                        for i in range(len(test_import['flat_ner'])):
                            ner = test_import['flat_ner'][i]
                            label = ner.nertag[0]
                            start = int(ner.start)
                            end = int(ner.end)
                            appendable_test_ner.append({"label": label, "start": start, "end": end})
                
            gold_ner.append(appendable_gold_ner)
            test_ner.append(appendable_test_ner)
        evaluator = Evaluator(gold_ner, test_ner, tags=['ORG', 'PER', 'MISC', 'LOC', 'LOC_ORG'])
        results, results_per_tag = evaluator.evaluate()
        all_results[subdistribution_for_testing] = (results, results_per_tag)
print("Programm on lõpetanud oma töö.")

Valmistan ette treenimistekste.
Treenimistekstid ette valmistatud 114.23665285110474 sekundiga.


Alustan nertaggeri treenimist.
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 347971
Seconds required: 1.402

Stochastic Gradient Descent (SGD)
c2: 0.001000
max_iterations: 1000
period: 10
delta: 0.000001

Calibrating the learning rate (eta)
calibration.eta: 0.100000
calibration.rate: 2.000000
calibration.samples: 1000
calibration.candidates: 10
calibration.max_trials: 20
Initial loss: 30999.990087
Trial #1 (eta = 0.100000): 4317.052042
Trial #2 (eta = 0.200000): 7494.701077
Trial #3 (eta = 0.400000): 13784.670564
Trial #4 (eta = 0.800000): 27677.081044
Trial #5 (eta = 1.600000): ERROR: overflow loss
nan (worse)
Trial #6 (eta = 0.050000): 3281.998028
Trial #7 (eta = 0.025000): 3066.810447
Trial #8 (eta = 0.012500): 3285.958481
Trial #9 (eta = 0.00625

***** Epoch #36 *****
Loss: 910.604066
Improvement ratio: 0.366425
Feature L2-norm: 86.109141
Learning rate (eta): 0.024955
Total number of feature updates: 584748
Seconds required for this iteration: 0.504

***** Epoch #37 *****
Loss: 872.108045
Improvement ratio: 0.395578
Feature L2-norm: 86.785024
Learning rate (eta): 0.024954
Total number of feature updates: 600991
Seconds required for this iteration: 0.495

***** Epoch #38 *****
Loss: 845.057402
Improvement ratio: 0.369127
Feature L2-norm: 87.441311
Learning rate (eta): 0.024953
Total number of feature updates: 617234
Seconds required for this iteration: 0.501

***** Epoch #39 *****
Loss: 825.777719
Improvement ratio: 0.359622
Feature L2-norm: 88.083879
Learning rate (eta): 0.024951
Total number of feature updates: 633477
Seconds required for this iteration: 0.498

***** Epoch #40 *****
Loss: 800.623362
Improvement ratio: 0.351267
Feature L2-norm: 88.709391
Learning rate (eta): 0.024950
Total number of feature updates: 649720
Seco

***** Epoch #79 *****
Loss: 423.046636
Improvement ratio: 0.126279
Feature L2-norm: 105.550833
Learning rate (eta): 0.024902
Total number of feature updates: 1283197
Seconds required for this iteration: 0.505

***** Epoch #80 *****
Loss: 418.530228
Improvement ratio: 0.122443
Feature L2-norm: 105.863536
Learning rate (eta): 0.024900
Total number of feature updates: 1299440
Seconds required for this iteration: 0.513

***** Epoch #81 *****
Loss: 411.669602
Improvement ratio: 0.138382
Feature L2-norm: 106.171334
Learning rate (eta): 0.024899
Total number of feature updates: 1315683
Seconds required for this iteration: 0.520

***** Epoch #82 *****
Loss: 413.817761
Improvement ratio: 0.101972
Feature L2-norm: 106.475168
Learning rate (eta): 0.024898
Total number of feature updates: 1331926
Seconds required for this iteration: 0.532

***** Epoch #83 *****
Loss: 402.438225
Improvement ratio: 0.148405
Feature L2-norm: 106.773321
Learning rate (eta): 0.024897
Total number of feature updates: 13

***** Epoch #119 *****
Loss: 308.584893
Improvement ratio: 0.044525
Feature L2-norm: 115.661448
Learning rate (eta): 0.024852
Total number of feature updates: 1932917
Seconds required for this iteration: 0.491

***** Epoch #120 *****
Loss: 299.864994
Improvement ratio: 0.075837
Feature L2-norm: 115.867514
Learning rate (eta): 0.024851
Total number of feature updates: 1949160
Seconds required for this iteration: 0.490

***** Epoch #121 *****
Loss: 299.754030
Improvement ratio: 0.058795
Feature L2-norm: 116.070471
Learning rate (eta): 0.024850
Total number of feature updates: 1965403
Seconds required for this iteration: 0.505

***** Epoch #122 *****
Loss: 297.867382
Improvement ratio: 0.076807
Feature L2-norm: 116.271904
Learning rate (eta): 0.024848
Total number of feature updates: 1981646
Seconds required for this iteration: 0.490

***** Epoch #123 *****
Loss: 294.740437
Improvement ratio: 0.077056
Feature L2-norm: 116.473266
Learning rate (eta): 0.024847
Total number of feature update

***** Epoch #162 *****
Loss: 239.062353
Improvement ratio: 0.056616
Feature L2-norm: 123.237141
Learning rate (eta): 0.024799
Total number of feature updates: 2631366
Seconds required for this iteration: 0.497

***** Epoch #163 *****
Loss: 238.367905
Improvement ratio: 0.049804
Feature L2-norm: 123.388960
Learning rate (eta): 0.024798
Total number of feature updates: 2647609
Seconds required for this iteration: 0.505

***** Epoch #164 *****
Loss: 240.687896
Improvement ratio: 0.038210
Feature L2-norm: 123.538581
Learning rate (eta): 0.024797
Total number of feature updates: 2663852
Seconds required for this iteration: 0.515

***** Epoch #165 *****
Loss: 237.536621
Improvement ratio: 0.046055
Feature L2-norm: 123.688160
Learning rate (eta): 0.024795
Total number of feature updates: 2680095
Seconds required for this iteration: 0.526

***** Epoch #166 *****
Loss: 237.840522
Improvement ratio: 0.040428
Feature L2-norm: 123.835297
Learning rate (eta): 0.024794
Total number of feature update

...

Täägitud fail Harju_Hageri_Kohila_id24217_1875a.json
Täägitud fail Tartu_V6nnu_Ahja_id17527_1885a.json
Täägitud fail P2rnu_Tori_Sindi_id11974_1885a.json
Täägitud fail Viljandi_P6ltsamaa_Adavere_id17828_1894a.json
Täägitud fail Tartu_Kodavere_Alatskivi_id14506_1876a.json
Täägitud fail Harju_Juuru_Juuru_id19472_1887a.json
Täägitud fail Harju_Kose_Palvere_id18184_1882a.json
Täägitud fail Tartu_Laiuse_Kivij2rve_id4917_1863a.json
Täägitud fail J2rva_Peetri_V2ike-Kareda_id21217_1875a.json
Täägitud fail J2rva_Tyri_S2revere_id14565_1886a.json
Täägitud fail V6ru_R2pina_R2pina_id10635_1868a.json
Täägitud fail Tartu_Kodavere_Alatskivi_id14511_1876a.json
Täägitud fail Tartu_Kodavere_Pala_id22058_1871a.json
Täägitud fail V6ru_Kanepi_Krootuse_id25390_1885a.json
Täägitud fail Tartu_V6nnu_Ahja_id14012_1882a.json
Täägitud fail Tartu_Kodavere_Alatskivi_id22167_1881a.json
Täägitud fail Tartu_Kodavere_Pala_id22543_1871a.json
Täägitud fail Viljandi_Paistu_Holstre_id9042_1836a.json
Täägitud fail Tartu_Kodav

Täägitud fail J2rva_Tyri_V22tsa_id16408_1885a.json
Täägitud fail J2rva_Tyri_Tyri-Alliku_id3994_1903a.json
Täägitud fail P2rnu_Audru_V6lla_id5150_1877a.json
Täägitud fail V6ru_R2pina_Kahkva_id8829_1888a.json
Täägitud fail L22ne_Karuse_Saastna_id22146_1867a.json
Täägitud fail Tartu_V6nnu_Ahja_id22857_1886a.json
Täägitud fail Tartu_V6nnu_Ahja_id12159_1873a.json
Täägitud fail Tartu_Torma_Avinurme_id5061_1860a.json
Täägitud fail Tartu_R6ngu_Aakre_id6659_1826a.json
Täägitud fail Tartu_V6nnu_Kiidj2rve_id24500_1866a.json
Täägitud fail Viru_Rakvere_Rakvere_id5143_1871a.json
Täägitud fail Tartu_V6nnu_Ahja_id17913_1885a.json
Täägitud fail Tartu_Kodavere_Pala_id23275_1872a.json
Täägitud fail L22ne_Reigi_K6rgessaare_id23087_1894a.json
Täägitud fail Tartu_N6o_Aru_id5374_1890a.json
Täägitud fail Harju_Juuru_Juuru_id20228_1868a.json
Täägitud fail Tartu_V6nnu_Rasina_id13313_1865a.json
Täägitud fail Tartu_V6nnu_Ahja_id15819_1883a.json
Täägitud fail Tartu_V6nnu_Ahja_id13002_1875a.json
Täägitud fail Tartu

***** Epoch #25 *****
Loss: 656.994614
Improvement ratio: 0.723579
Feature L2-norm: 96.389820
Learning rate (eta): 0.049875
Total number of feature updates: 390350
Seconds required for this iteration: 0.598

***** Epoch #26 *****
Loss: 647.638765
Improvement ratio: 0.613312
Feature L2-norm: 97.332915
Learning rate (eta): 0.049870
Total number of feature updates: 405964
Seconds required for this iteration: 0.604

***** Epoch #27 *****
Loss: 620.109720
Improvement ratio: 0.616911
Feature L2-norm: 98.253606
Learning rate (eta): 0.049865
Total number of feature updates: 421578
Seconds required for this iteration: 0.602

***** Epoch #28 *****
Loss: 595.886809
Improvement ratio: 0.574674
Feature L2-norm: 99.121473
Learning rate (eta): 0.049860
Total number of feature updates: 437192
Seconds required for this iteration: 0.600

***** Epoch #29 *****
Loss: 581.522826
Improvement ratio: 0.487664
Feature L2-norm: 99.962799
Learning rate (eta): 0.049855
Total number of feature updates: 452806
Seco

***** Epoch #69 *****
Loss: 273.931712
Improvement ratio: 0.116081
Feature L2-norm: 120.754217
Learning rate (eta): 0.049657
Total number of feature updates: 1077366
Seconds required for this iteration: 0.616

***** Epoch #70 *****
Loss: 269.671840
Improvement ratio: 0.243345
Feature L2-norm: 121.095540
Learning rate (eta): 0.049652
Total number of feature updates: 1092980
Seconds required for this iteration: 0.650

***** Epoch #71 *****
Loss: 296.260628
Improvement ratio: 0.002884
Feature L2-norm: 121.438184
Learning rate (eta): 0.049648
Total number of feature updates: 1108594
Seconds required for this iteration: 0.653

***** Epoch #72 *****
Loss: 276.279478
Improvement ratio: 0.062100
Feature L2-norm: 121.771645
Learning rate (eta): 0.049643
Total number of feature updates: 1124208
Seconds required for this iteration: 0.633

***** Epoch #73 *****
Loss: 281.747648
Improvement ratio: 0.088412
Feature L2-norm: 122.099274
Learning rate (eta): 0.049638
Total number of feature updates: 11

Täägitud fail Harju_Juuru_Juuru_id20490_1869a.json
Täägitud fail P2rnu_Halliste_Penuja_id416_1885a.json
Täägitud fail Tartu_Kodavere_Alatskivi_id2041_1878a.json
Täägitud fail L22ne_Ridala_Sinalepa_id24333_1882a.json
Täägitud fail Harju_Kose_Kose-Uuem6isa_id5292_1869a.json
Täägitud fail V6ru_R6uge_Leevi_id24491_1875a.json
Täägitud fail Harju_Rapla_Rapla_id19152_1870a.json
Täägitud fail L22ne_Pyhalepa_Kassari_id23159_1867a.json
Täägitud fail Harju_Kose_Palvere_id13236_1879a.json
Täägitud fail Tartu_Kodavere_Pala_id20761_1867a.json
Täägitud fail J2rva_Peetri_V2ike-Kareda_id20031_1872a.json
Täägitud fail Tartu_V6nnu_Ahja_id9655_1871a.json
Täägitud fail Tartu_Kodavere_Pala_id21194_1868a.json
Täägitud fail Harju_J6el2htme_J6el2htme_id8154_1888a.json
Täägitud fail Tartu_V6nnu_Ahja_id15090_1883a.json
Täägitud fail Harju_Kose_Palvere_id24675_1872a.json
Täägitud fail L22ne_Kullamaa_Piirsalu_id7871_1890a.json
Täägitud fail Tartu_Otep22_Pyhaj2rve_id4314_1885a.json
Täägitud fail Tartu_V6nnu_Ahja_id

In [57]:
gold_ner = []
test_ner = []

for subdistribution in [1, 2, 3, 4, 5]:
    training_subdistributions = []
    for y in [1, 2, 3, 4, 5]:
        if y == subdistribution:
            subdistribution_for_testing = y
        else:
            training_subdistributions.append(y)
    

    for file in {key: value for key, value in files.items() if int(value) == subdistribution_for_testing}:
        appendable_gold_ner = []
        appendable_test_ner = []

        if file.endswith(".json"):
            if file in files_not_working:
                continue
            else:
                with open("./vallakohtufailid-trained-nertagger/" + str(file), 'r', encoding='UTF-8') as f_test, \
                    open("./vallakohtufailid-json-flattened/" + str(file), 'r', encoding='UTF-8') as f_gold:
                        test_import = json_to_text(f_test.read())
                        gold_import = json_to_text(f_gold.read())

                        # The commented part is needed for word-level-ner.
                        '''
                        for i in range(len(gold_import['flat_gold_wordner'])):
                            tag = gold_import['flat_gold_wordner'][i].nertag[0]
                            gold.append(tag)
                        for i in range(len(test_import['flat_wordner'])):
                            tag = test_import['flat_wordner'][i].nertag[0]
                            test.append(tag)
                        '''

                        for i in range(len(gold_import['gold_ner'])):
                            ner = gold_import['gold_ner'][i]
                            label = ner.nertag
                            start = int(ner.start)
                            end = int(ner.end)
                            appendable_gold_ner.append({"label": label, "start": start, "end": end})

                        for i in range(len(test_import['flat_ner'])):
                            ner = test_import['flat_ner'][i]
                            label = ner.nertag[0]
                            start = int(ner.start)
                            end = int(ner.end)
                            appendable_test_ner.append({"label": label, "start": start, "end": end})
                
            gold_ner.append(appendable_gold_ner)
            test_ner.append(appendable_test_ner)
print("Programm on lõpetanud oma töö.")

Programm on lõpetanud oma töö.


In [52]:
with open("results_new.txt", "w+") as results_file:
    results_file.write(json.dumps(all_results))

In [3]:
with open("results_new.txt", "r") as f:
    results_json = json.loads(f.read())

### Tulemused alamhulkade kaupa:

In [43]:
correct_all = 0
actual_all = 0
possible_all = 0
df = dict()

for i in ['1', '2', '3', '4', '5']:
    train = []
    for j in ['1', '2', '3', '4', '5']:
        if j == i:
            subdistribution_for_testing = j
        else:
            train.append(j)
    correct = results_json[i][0]['strict']['correct']
    correct_all += correct
    actual = results_json[i][0]['strict']['actual']
    actual_all += actual
    possible = results_json[i][0]['strict']['possible']
    possible_all += possible
    precision = (correct / actual)
    recall = (correct / possible)
    f1 = 2 * ((precision * recall) / (precision + recall))
    df[str(subdistribution_for_testing)] = [precision, recall, f1]

precision = correct_all / actual_all
recall = correct_all / possible_all
f1 = 2 * ((precision * recall) / (precision + recall))
df["Total"] = [precision, recall, f1]

dataframe = pd.DataFrame(df, index=["Precision", "Recall", "F1-score"])
dataframe.columns.name = "Alamhulk"
display(dataframe)

Alamhulk,1,2,3,4,5,Total
Precision,0.911927,0.914806,0.905153,0.882717,0.911048,0.904403
Recall,0.879646,0.889086,0.880174,0.847297,0.881895,0.874555
F1-score,0.895495,0.901763,0.892489,0.864645,0.896234,0.889228


### Tulemused nimeüksuste liigi kaupa:

In [46]:
df = dict()
for i in ['1', '2', '3', '4', '5']:
    train = []
    by_kind = dict()
    for j in ['1', '2', '3', '4', '5']:
        if j == i:
            subdistribution_for_testing = j
        else:
            train.append(j)
            
    for key in list(results_json[i][1].keys()):
        correct = results_json[i][1][str(key)]['strict']['correct']
        actual = results_json[i][1][str(key)]['strict']['actual']
        possible = results_json[i][1][str(key)]['strict']['possible']
        precision = (correct / actual)
        recall = (correct / possible)
        f1 = 2 * ((precision * recall) / (precision + recall))
        by_kind[str(key) + "_precision"] = precision
        by_kind[str(key) + "_recall"] = recall
        by_kind[str(key) + "_f1score"] = f1
    df[str(subdistribution_for_testing)] = by_kind

display(pd.DataFrame(df))

,1,2,3,4,5
ORG_precision,0.777778,0.777778,0.848485,0.764706,0.728395
ORG_recall,0.711864,0.742424,0.800000,0.712329,0.756410
ORG_f1score,0.743363,0.759690,0.823529,0.737589,0.742138
PER_precision,0.939442,0.944829,0.939235,0.908825,0.946711
PER_recall,0.927235,0.928010,0.928118,0.905157,0.938812
PER_f1score,0.933298,0.936344,0.933643,0.906987,0.942745
MISC_precision,0.700000,0.868421,0.743590,0.756757,0.631579
MISC_recall,0.512195,0.647059,0.690476,0.800000,0.600000
MISC_f1score,0.591549,0.741573,0.716049,0.777778,0.615385
LOC_precision,0.533981,0.650602,0.641667,0.556391,0.666667


### Confusion matrix

In [135]:
uus_gold_ner = []
uus_test_ner = []

for i in range(len(gold_ner)):
    for j in range(len(test_ner[i])):
        element_test = test_ner[i][j]
        for element_gold in gold_ner[i]:
            if element_test['start'] == element_gold['start'] and element_test['end'] == element_gold['end']:
                uus_gold_ner.append(element_gold)
                uus_test_ner.append(element_test)

In [136]:
y_true = pd.Series([x['label'] for x in uus_gold_ner], name="Actual")
y_pred = pd.Series([x['label'] for x in uus_test_ner], name="Predicted")

In [137]:
pd.crosstab(y_true, y_pred)

Predicted,LOC,LOC_ORG,MISC,ORG,PER
Actual,,,,,
LOC,406,58,0,0,43
LOC_ORG,58,1501,2,1,87
MISC,2,2,135,0,10
ORG,0,4,0,258,0
PER,7,23,1,0,18078
